In [1]:
!pip3 install pandas-gbq -U
!pip3 install pandas
!pip3 install pandasql
!pip3 install google-cloud-bigquery --upgrade
#!pip3 install google-cloud-python

Requirement already up-to-date: pandas-gbq in /usr/local/envs/py3env/lib/python3.5/site-packages (0.9.0)
Requirement already up-to-date: google-cloud-bigquery in /usr/local/envs/py3env/lib/python3.5/site-packages (1.8.1)


In [2]:
from datalab.context import Context
import datalab.storage as storage
import datalab.bigquery as bq
import pandas as pd


project = Context.default().project_id
# Years list for the extraction
ylist = [year for year in range(2005,2019)]

query = " select closest.id, closest.name, closest.latitude, closest.longitude, element as stats_name, " \
        "  value as stats_value, date as measure_date " \
        " from `bigquery-public-data.ghcn_d.ghcnd_{}` as data right join (  " \
             " select id, name, state, latitude, longitude, " \
             "  ST_DISTANCE(ST_GEOGPOINT(longitude, latitude),ST_GEOGPOINT( -73.561667, 45.508889 )) / 1000.0 as distance  " \
             "  from `bigquery-public-data.ghcn_d.ghcnd_stations`  " \
             " where longitude is not null and latitude is not null) " \
             " as closest on data.id = closest.id " \
        " where  extract(year from data.date) =  {} " \
        " and closest.distance <= 20 and  element in ('TAVG','PRCP')  "


dflist = []

for pullingyear in ylist:
  bqu = query.format(pullingyear,pullingyear)
  df = pd.read_gbq(bqu,project_id=project,dialect='standard')
  dflist.append(df)
  print("the query for the year {} has been completed successfully".format(pullingyear))
  
  
dfcompiled = pd.concat(dflist)

dfcompiled.to_gbq('firemandb.hst_tmp_weather', 
                 Context.default().project_id,
                 chunksize=10000, 
                 if_exists='append',
                 verbose=False
                 )
  


the query for the year 2005 has been completed successfully
the query for the year 2006 has been completed successfully
the query for the year 2007 has been completed successfully
the query for the year 2008 has been completed successfully
the query for the year 2009 has been completed successfully
the query for the year 2010 has been completed successfully
the query for the year 2011 has been completed successfully
the query for the year 2012 has been completed successfully
the query for the year 2013 has been completed successfully
the query for the year 2014 has been completed successfully
the query for the year 2015 has been completed successfully
the query for the year 2016 has been completed successfully
the query for the year 2017 has been completed successfully
the query for the year 2018 has been completed successfully
